# GMM

## Load Packages

In [1]:
using Optim
using Compat, Missings        #in Julia 0.6 
#using Dates, DelimitedFiles   #in Julia 0.7

include("jlFiles/printmat.jl")
include("jlFiles/NWFn.jl")

NWFn

# GMM I

This section describes the basic (exactly identified) GMM, that is, when we have as many moment conditions as parameters. (In this case GMM is the same as the classical method of moments, MM.)

The first few cells load functions and data. See further down for the estimation.

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)   #start on line 2, column 1
x = x[:,2]         #excess market returns, in %

T  = size(x,1)

388

## Traditional Estimation of Mean and Variance

The next cell applies the traditional way of estimating the mean and the variance.

In [3]:
μ  = mean(x)                      #same as setting A*gbar=0
σ² = var(x,corrected=false)       #"false" to use 1/T formula

par_a = [μ;σ²]

println("\nParameters (col 1) and traditional std(parameters) in col 2:")
printmat([par_a [sqrt((σ²/T));sqrt(2*σ²^2/T)]])


Parameters (col 1) and traditional std(parameters) in col 2:
     0.602     0.233
    21.142     1.518



## GMM Point Estimates and Distribution

To estimate the mean and variance of $x_{t}$, use the following moment condition

$
\frac{1}{T}\sum\nolimits_{t=1}^{T}g_{t}=0 \: \text{ where } 
$

$
g_{t}(\mu,\sigma^{2})=\left[
\begin{array}
[c]{l}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}
\end{array}
\right] .
$

The distribution of the estimates is

$
\sqrt{T}(\hat{\mu}-\mu_{0})\overset{d}{\rightarrow}N(0,V) 
\: \text{ where } \: 
V = (D_{0}^{\prime}S_{0}^{-1}D_{0})  ^{-1}
$

Clearly, $D_{0}=-\textrm{I}$ and if data is iid then $S_{0}=\mathrm{Var}(g_{t})$.

In [4]:
function Gmm2MomFn(par,x)
    (μ,σ²) = (par[1],par[2])
    g      = [(x .- μ) ((x .- μ).^2 .- σ²)]  #Tx2         
    gbar   = vec(Compat.mean(g,dims=1))      #2-element vector, 0.7 syntax
    return g,gbar
end

Gmm2MomFn (generic function with 1 method)

In [5]:
(g,gbar) = Gmm2MomFn(par_a,x)           #Tx2, moment conditions
println("Checking if mean of g_t = 0")
printmat(gbar)

D  = [-1   0;                #Jacobian
       0  -1]
S  = NWFn(g,1)
V1 = inv(D'inv(S)*D)

println("\nparameter, std(parameters)")
printmat([par_a sqrt.(diag(V1/T))])

Checking if mean of g_t = 0
    -0.000
     0.000


parameter, std(parameters)
     0.602     0.244
    21.142     2.381



# GMM II

This section expands the GMM calculations by doing an overidentified case: more moment conditions than parameters.

Warning: some of the variables (```g,S,etc```) are overwritten with new values.

## The Moment Conditions

If $x_{t}$ is $N(\mu,\sigma^{2})$, then the following moment conditions should
all be zero (in expectation)

$
g_{t}(\mu,\sigma^{2})=\left[
\begin{array}
[c]{l}
x_{t}-\mu\\
(x_{t}-\mu)^{2}-\sigma^{2}\\
(x_{t}-\mu)^{3}\\
(x_{t}-\mu)^{4}-3\sigma^{4}
\end{array}
\right]  .
$

The first moment condition defines the mean $\mu$, the second defines the
variance $\sigma^{2}$, while the third and forth are the skewness and excess
kurtosis respectively.

In [6]:
function Gmm4MomFn(par,x)
  (μ,σ²) = (par[1],par[2])
  g      = [(x .- μ) ((x .- μ).^2 .- σ²) ((x .- μ).^3) ((x .- μ).^4 .- 3*σ²^2)]    #Tx4
  gbar   = vec(Compat.mean(g,dims=1))     #4-element vector, 0.7 syntax
  return g,gbar
end


function DGmm4MomFn(par,x)
    (μ,σ²) = (par[1],par[2])
    D  = [-1                  0    ;                #Jacobian of Gmm4MomFn
          -2*mean(x.-μ)      -1    ;
          -3*mean((x.-μ).^2)   0   ;
          -4*mean((x.-μ).^3)  -6*σ²]
    return D
end    

DGmm4MomFn (generic function with 1 method)

## GMM: A*g = 0


The following code from estimates the parameters (mean and
variance) by combining the 4 original moment conditions in $\bar{g}$ into 2
effective moment conditions, $A\bar{g}$, where $A$ is a $2\times4$ matrix

$
A=\left[
\begin{array}
[c]{cccc}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0
\end{array}
\right]  
$ 

This particular $A$ matrix implies that we use the classical
estimators of the mean and variance.

In [7]:
(g,gbar) = Gmm4MomFn(par_a,x)        #Tx4, moment conditions. Warning: overwriting old g
q = size(g,2)

A = [1 0 0 0;                       #A in A*gbar=0 (here: all weight on first two moments)
     0 1 0 0]
println("\nChecking if mean of A*g_t = 0")
printmat(A*gbar)

D  = DGmm4MomFn(par_a,x)               #Jacobian
println("\nThe Jacobian is:")
printmat(D)

S  = NWFn(g,1)
V3 = inv(A*D)*A*S*A'inv(A*D)'

println("\nparameter, std(parameters)")
printmat([par_a sqrt.(diag(V3/T))])


Checking if mean of A*g_t = 0
    -0.000
     0.000


The Jacobian is:
    -1.000     0.000
     0.000    -1.000
   -63.427     0.000
   314.797  -126.854


parameter, std(parameters)
     0.602     0.244
    21.142     2.381



## GMM: Minimizing gbar'W*gbar


The following code applies a numerical method to solve a minimization problem with the weighting matrix 

$
W=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0
\end{bmatrix}
$

The results should be the same (or at least very close to) the previous results, since the $W$ matrix puts all weight on the first two moments (basically mimicking the estimations above).

The first step is to define a loss function as 

$
\bar{g}'W\bar{g}
$

As a practical matter, it is often the case that a derivative-free method works better than other optimization routines.

In [8]:
function Gmm4MomLossFn(par,x,W=1)
  (g,gbar) = Gmm4MomFn(par,x)
  Loss     = 1.0 + gbar'W*gbar      #to be minimized
  return Loss
end

Gmm4MomLossFn (generic function with 2 methods)

In [9]:
println("\nGMM with weighting matrix")
                                          #gbar'W*gbar
W     = Compat.diagm(0=>[1;1;0;0])        #weighting matrix, 0.7 syntax
Sol   = optimize(par->Gmm4MomLossFn(par,x,W),par_a)

par_b = Optim.minimizer(Sol)

g,    = Gmm4MomFn(par_b,x)                #Tx4, moment conditions, evaluated at point estimates
S     = NWFn(g,1)                         #variance of sqrt(T)"gbar, NW with 1 lag
V2    = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)

println("Weighting matrix")
printmat(W)
println("\nparameter, std(parameters)")
printmat([par_b sqrt.(diag(V2/T))])


GMM with weighting matrix
Weighting matrix
         1         0         0         0
         0         1         0         0
         0         0         0         0
         0         0         0         0


parameter, std(parameters)
     0.602     0.244
    21.142     2.381



# GMM: Minimizing g'Wg, Iterating over W


The following code iterates over the weighting matrix by using $W=S^{-1}$, where

$S = \text{Cov}(\sqrt{T}\bar{g})$ 

is from the previous iteration.

In [10]:
println("\niterated GMM, using optimal weighting matrix, starting with S from previous estimation")

(par_c,par0) = (copy(par_b),copy(par_b))
Δpar  = Inf
i     = 1
println("\n\niterating over W starting with the W choice above")
while (Δpar > 1e-3) || (i < 2)    #require at least one iteration
    global Dpar, par_c, par0, i, W, S    #to change globals from inside loop
    local Sol, g
    println("-------iteration  $i, old and new parameters--------")
    W               = inv(S)
    Sol             = optimize(par->Gmm4MomLossFn(par,x,W),par0)   #use last estimates as starting point
    par_c           = Optim.minimizer(Sol)
    printlnPs(par0')
    printlnPs(par_c')
    g,              = Gmm4MomFn(par_c,x)
    S               = NWFn(g,1)
    Δpar            = maximum(abs.(par_c-par0))
    par0            = copy(par_c)             #par0=par_c would make them always identical
    i               = i + 1
 end

V2 = inv(D'W*D)*D'W*S*W'D*inv(D'W*D)      #if non-optimal weighting matrix
V1 = inv(D'inv(S)*D)                      #with optimal weighting matrix

println("\nparameter   std_ver2  std_ver1")
printmat([par_c sqrt.(diag(V2/T)) sqrt.(diag(V1/T))])


iterated GMM, using optimal weighting matrix, starting with S from previous estimation


iterating over W starting with the W choice above
-------iteration  1, old and new parameters--------
     0.602    21.142
     0.877    16.916
-------iteration  2, old and new parameters--------
     0.877    16.916
     0.879    16.648
-------iteration  3, old and new parameters--------
     0.879    16.648
     0.879    16.645
-------iteration  4, old and new parameters--------
     0.879    16.645
     0.879    16.647
-------iteration  5, old and new parameters--------
     0.879    16.647
     0.879    16.647

parameter   std_ver2  std_ver1
     0.879     0.217     0.217
    16.647     1.311     1.311

